#Papers
Manuals:<br>
https://radimrehurek.com/gensim/auto_examples/index.html#documentation
https://pyldavis.readthedocs.io/en/latest/index.html

Tutorials:<br>
https://blog.codecentric.de/2017/01/topic-modeling-codecentric-blog-articles/
https://www.tutorialspoint.com/gensim/gensim_creating_lda_topic_model.htm

Papers:<br>
Blei et al.: https://jmlr.org/papers/volume3/blei03a/blei03a.pdf<br>
Thomsen/Mimno: https://arxiv.org/abs/2010.12626<br>
Sievert/Shirley: http://nlp.stanford.edu/events/illvi2014/papers/sievert-illvi2014.pdf<br> 


# Einführung

Sie befinden sich nun in einer Programmierkonsole, die auf einer virtuellen Maschine, also nicht auf Ihrem Rechner ausgeführt wird. In den folgenden "Zellen" wechseln sich Textbausteine und Programmierzellen ab, in denen Sie Programmcode ausführen können. Alles ist soweit vorbereitet, dass Sie nur in der oberen linken Ecke einer Zelle zwischen den eckigen Klammern -> [  ] das Play-Symbol klicken müssen, das erscheint, wenn Sie mit der Maus über die Klammern fahren. Die Zellen müssen strikt in der Reihenfolge von oben nach unten ausgeführt werden, da spätere Zellen auf die Variablen voriger aufbauen. In einigen Zellen finden Sie auf der rechten Seite Parameter, die Sie einstellen können.


# Set up System

In dieser Sektion wird das Colab mit dem Google Drive verknüpft, um Daten einzulesen und speichern zu können.

## Mount Drive Folders

Hier wird das Google Drive als virtuelles Laufwerk eingebunden.

In [ ]:
root = "/content/gdrive"
from google.colab import drive
drive.mount(root, force_remount=True)

Mounted at /content/gdrive


##Set Paths

Hier werden Pfade zu Dateiverzeichnissen definiert.

In [ ]:
import os

for folder in os.listdir(root+"/My Drive/"):
  if folder.startswith("Data MuM"):
    if '_init_mum.txt' in os.listdir(root+"/My Drive/"+folder):
      path_name = folder+'/'

try:
  path = root+"/My Drive/"+path_name

  data_path = path
  output_path = root+"/My Drive/"+"WS MuM OUTPUT/"

  print("data_path: "+data_path)
  print("output_path: "+output_path)

  try:
    os.mkdir(output_path)
    print ("Successfully created the directory %s " % output_path)
  except OSError:
    print ("Directory %s already exists" % output_path)
except NameError:
  print('Ordner Data MuM konnte nicht gefunden werden. Ist der Ordner mit dem Drive verknüpft?')




data_path: /content/gdrive/My Drive/Data MuM/
output_path: /content/gdrive/My Drive/WS MuM OUTPUT/
Directory /content/gdrive/My Drive/WS MuM OUTPUT/ already exists


##Import Global Packages

Hier werden Ordner angelegt bzw. überprüft, ob bereits Ordner angelegt wurden. Anschließend werden zusätzliche Bibliotheken der zugrundliegenden Programmiersprache Python importiert, die für das Topic Modeling erforderlich sind.

In [ ]:
!pip install matplotlib
!pip install numexpr
!pip install xlsxwriter 
import os
import re
from datetime import datetime
from pprint import pprint
import pandas as pd
from matplotlib import pylab
import matplotlib.pyplot as plt
import seaborn as sns
from seaborn.matrix import heatmap
import plotly.express as px



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 8.0 MB/s 


#Fertig berechnete Modelle laden
Hier können gespeicherte Modelle geladen werden. Sämtliche Analyse- und Evaluationsfunktionen können mit den geladenen Daten genutzt werden. Das ermöglicht den schnellen Vergleich einer großen Anzahl verschiedener Modelle. Wenn Sie die Zelle ausführen, erscheint unter der Zelle eine Liste mit Dateinamen. Markieren Sie eine Datei durch Doppelklick und fügen diese in das Textfeld darunter ein. Bestätigen Sie mit Enter.

In [ ]:
modeldumps = data_path+"OUTPUT/modeldumps/"

#@markdown Die Interviews müssen, um optimal und feingranular modelliert zu werden, \
#@markdown in Chunks unterteilt werden. Wählen Sie zwischen der harten Unterteilung \
#@markdown nach Sätzen (df_sentences) oder nach Sprecherwechseln (df_speakers) [DEFAULT]. \
#@markdown 
chunk_type = 'df_speakers' #@param ['df_speakers', 'df_sentences']


imports = [file for file in os.listdir(modeldumps) if 'mallet' in file and chunk_type in file]
for file in imports:
  print(file)

path_model_mallet = input('\n\nMallet-Modell aus Liste hier hinein kopieren')
df_import_doc_tops_mallet = pd.read_pickle(modeldumps+path_model_mallet+'/'+path_model_mallet+'.doc_tops_mallet')
doc_tops_mallet = df_import_doc_tops_mallet.values.tolist()
df_import_top_words_mallet = pd.read_pickle(modeldumps+path_model_mallet+'/'+path_model_mallet+'.top_words_mallet')
top_words_mallet = df_import_top_words_mallet.values.tolist()
settings = open(modeldumps+path_model_mallet+'/'+path_model_mallet+'.txt', mode='r', encoding='utf-8').read()

file = settings.splitlines()[0]
dataframeimport = pd.read_pickle(path+file)  
load_dataset_mallet = dataframeimport.values.tolist()
name_dataset_mallet = file
import_reference_dataset_mallet = pd.read_pickle(path+file.split(' - ')[0] + ' - REFERENCE')
reference_dataset_mallet = import_reference_dataset_mallet.values.tolist()
for line in settings.splitlines():
  if line.startswith('Anzahl Topics'):
    topics_mallet = int(line.split(': ')[1])
print(topics_mallet)

# Topic-Weights berechnen


sum_top_weights = 0.0
top_counter = 0
min_weight_mallet = 1
max_weight_mallet = 0
for line in doc_tops_mallet:
  for tup in line:
      sum_top_weights = sum_top_weights + float(tup[1])
      top_counter += 1
      if float(tup[1]) < min_weight_mallet:
        min_weight_mallet = float(tup[1])
      if float(tup[1]) > max_weight_mallet:
        max_weight_mallet = float(tup[1])

average_weight_mallet = sum_top_weights/top_counter

print('Minimales Topic-Weight Mallet: ' + str(min_weight_mallet))
print('Durchschnittliches Topic-Weight Mallet: ' + str(average_weight_mallet))
print('Maximales Topic-Weight Mallet: ' + str(max_weight_mallet))

load_models = True

mallet_LUSIR_df_speakers_clean_normalized_50sentence(s)_NEW - CORPUS_50topics_2022-04-29 11_54_52
mallet_LUSIR_df_speakers_clean_normalized_25sentence(s)_NEW - CORPUS_50topics_2022-04-29 11_41_03
mallet_LUSIR_df_speakers_clean_normalized_10sentence(s)_NEW - CORPUS_50topics_2022-04-29 07_39_11
mallet_LUSIR_df_speakers_clean_normalized_5sentence(s)_NEW - CORPUS_50topics_2022-04-29 06_51_34
mallet_LUSIR_df_speakers_clean_normalized_1sentence(s)_NEW - CORPUS_50topics_2022-04-29 06_18_12


Mallet-Modell aus Liste hier hinein kopierenmallet_LUSIR_df_speakers_clean_normalized_25sentence(s)_NEW - CORPUS_50topics_2022-04-29 11_41_03
50
Minimales Topic-Weight Mallet: 0.00017526468550765437
Durchschnittliches Topic-Weight Mallet: 0.020000000000004278
Maximales Topic-Weight Mallet: 0.8757353117624125


#Topics ausgeben

Hier werden die Top-Keywords aller Topics als einfache Liste ausgegeben. Sie können die Anzahl der jeweiligen Keywords rechts angeben. Mit >save_fig< können Sie die Liste als txt-Datei in den Ordner >WS MuM OUTPUT< speichern, den Sie in Ihrem Google Drive finden.

In [ ]:
#@markdown ##Wie viele Keywords pro Topic sollen angezeigt werden?

number_of_words =  15#@param {type:'integer'}

#@markdown ##Keywords als Textdatei speichern?
save_doc = False #@param {type:"boolean"}

now = str(datetime.now())[:19]

if load_models == False:
  output = lda_model_mallet.print_topics(num_topics=topics_mallet, num_words=number_of_words)

  if save_doc:
    out = open(output_path+'Top' + str(number_of_words) + '_keywords_mallet_'+name_dataset_mallet[:-20]+'_'+str(topics_mallet)+'topics_'+ now +'.txt', 'w', encoding='UTF-8')
    for line in output:
      out_line = str(int(line[0])) + ' ' + str(re.findall(r"\"(.*?)\"", line[1]))+'\n'
      out.write(out_line)
      print(out_line)
    out.close()

  else:
    for line in output:
      out_line = str(int(line[0])) + ' ' + str(re.findall(r"\"(.*?)\"", line[1]))+'\n'
      print(out_line)

if load_models == True:
  if save_doc:
    out = open(output_path+'Top' + str(number_of_words) + '_keywords_mallet_'+name_dataset_mallet[:-20]+'_'+str(topics_mallet)+'topics_'+ now +'.txt', 'w', encoding='UTF-8')
    for line in top_words_mallet:
      newline = []
      for i in range (0, number_of_words):
        newline.append(line[1].split(' + ')[i])
      out_line = str(int(line[0])) + ' ' + str(re.findall(r"\"(.*?)\"", str(newline)))+'\n'
      out.write(out_line)
      print(out_line)
    out.close()

  else:
    for line in top_words_mallet:
      newline = []
      for i in range (0, number_of_words):
        newline.append(line[1].split(' + ')[i])
      out_line = str(int(line[0])) + ' ' + str(re.findall(r"\"(.*?)\"", str(newline)))+'\n'
      print(out_line)

0 ['deutsch', 'amerikaner', 'deutschland', 'engländer', 'franzosen', 'englisch', 'russen', 'deutschen', 'polen', 'frankreich', 'französisch', 'england', 'amerika', 'amerikanisch', 'soldaten']

1 ['geld', 'verdienen', 'bezahlen', 'kaufen', 'kosten', 'pfennig', 'zahlen', 'rente', 'sparen', 'währungsreform', 'verkaufen', 'abgeben', 'sachen', 'ausgeben', 'teuer']

2 ['chef', 'büro', 'angestellt', 'abteilung', 'thyssen', 'sekretärin', 'abteilungen', 'arbeit', 'herren', 'damen', 'einstellen', 'personalabteilung', 'hütte', 'kollegen', 'beschäftigen']

3 ['morgens', 'abends', 'essen', 'kochen', 'waschen', 'putzen', 'woche', 'mittags', 'sonntags', 'wäsche', 'samstags', 'schlafen', 'arbeit', 'nachmittags', 'aufstehen']

4 ['politisch', 'meinung', 'menschen', 'diskutieren', 'politik', 'politische', 'positiv', 'jugend', 'reden', 'rolle', 'mensch', 'sinne', 'nationalsozialismus', 'erleben', 'gefühl']

5 ['juden', 'gewusst', 'jude', 'wussten', 'passieren', 'nazis', 'jüdische', 'jüdisch', 'krieg', 'm

# Analyse & Evaluation

## Doc-Top Heatmap 

Diese Heatmap ist eine Interview-Topic-Matrix. Je dunkler, desto mehr Gewicht hat ein Topic in einem Interview. Der matrix_type "chunk_top" zeigt eine Textabschnitt-Topic-Matrix, die allerdings extreme Werte auf der Y-Achse aufweist. Mit >save_fig< können Sie die Heatmap in den Ordner >WS MuM OUTPUT< speichern, den Sie in Ihrem Google Drive finden.

In [ ]:
#@markdown Soll nur eine Auswahl von Interviews angezeigt werden? IDs in Textfeld eintragen, kommasepariert.
document_search = False #@param {type:"boolean"}
document_id = "ADG0001,ADG0002,ADG0003" #@param {type:"string"}
document_ids_repl = document_id.replace(' ', '')
document_ids = [id for id in document_ids_repl.split(',')]

#@markdown ##Grafik speichern?
save_fig = False #@param {type:"boolean"}

#@markdown ##Größe Grafik (in Pixeln)
breite = 1000 #@param {type:"integer"}
höhe = 800 #@param {type:"integer"}

now = str(datetime.now())[:19]
matrix_type = 'doc_top'

# Keywords für X-Achse
if load_models == False:
  output = lda_model_mallet.print_topics(num_topics=topics, num_words=10)
  

if load_models == True:
  raw_data = load_dataset_mallet
  output = []
  for line in top_words_mallet:
    newline = ''
    for i in range (0, 10):
      newline = newline + line[1].split(' + ')[i] + ' '
    output.append([line[0], newline])

keywords_mallet = []

for line in output:
  out_line = line[0],str(re.findall(r"\"(.*?)\"", line[1]))
  keywords_mallet.append(out_line)


if matrix_type == 'doc_top':

    interview_id = raw_data[0][0].split('_')[0]
    interview_topics = {}
    tops_per_int_counter = []
    chunk_count = 1

    for i, line in enumerate(doc_tops_mallet):
      if interview_id == raw_data[i][0].split('_')[0]:
          for tup in line:
            if tup[0] not in interview_topics:
              interview_topics[tup[0]] = tup[1]
            if tup[0] in interview_topics:
              interview_topics[tup[0]] += tup[1]
          chunk_count += 1

      if interview_id != raw_data[i][0].split('_')[0] or i == len(raw_data)-1:
        for topic, count in interview_topics.items():
            tops_per_int_counter.append([interview_id, topic, count/chunk_count])
        interview_id = raw_data[i][0].split('_')[0]
        interview_topics = {}
        chunk_count = 1
        for tup in line:
            if tup[0] not in interview_topics:
              interview_topics[tup[0]] = tup[1]
            if tup[0] in interview_topics:
              interview_topics[tup[0]] += tup[1]

if document_search:
  transfer = []
  for line in tops_per_int_counter:
    if line[0].split('_')[0] in document_ids:
      transfer.append(line)
  tops_per_int_counter = transfer


df_heatmap = pd.DataFrame (tops_per_int_counter, columns = ['doc', 'top', 'weight'])
doc_tops_heatmap = df_heatmap.pivot("doc", "top", "weight")
fig = px.imshow(doc_tops_heatmap, color_continuous_scale = 'emrld', height=höhe, width=breite, aspect='auto')
fig.show()


## Topics im Interviewverlauf

Diese Heatmap zeigt Topics und deren Gewichtung im chronologischen Verlauf des interviews. Auf diese Weise können Themenwechsel und -überlagerungen lokalisiert werden. Wählen Sie das gewünschte Interview rechts aus. Mit >save_fig< können Sie die Heatmap in den Ordner >WS MuM OUTPUT< speichern, den Sie in Ihrem Google Drive finden.

#### interaktive Version

In [ ]:
#@markdown Interview ID (suche nach ADG0001 etc. - vgl. Y-Achse der Doc-Top-Heatmap )

interview_search_id = 'ADG0089' #@param {type: 'string'}

#@markdown ##Grafik speichern?
save_fig = False #@param {type:"boolean"}
now = str(datetime.now())[:19]

#@markdown ##Größe Grafik (in Pixeln)
breite = 1500 #@param {type:"integer"}
höhe = 1000 #@param {type:"integer"}

if load_models == False:
  output = lda_model_mallet.print_topics(num_topics=topics, num_words=10)
  

if load_models == True:
  raw_data = load_dataset_mallet
  output = []
  for line in top_words_mallet:
    newline = ''
    for i in range (0, 10):
      newline = newline + line[1].split(' + ')[i] + ' '
    output.append([line[0], newline])

keywords_mallet = []

for line in output:
  out_line = line[0],str(re.findall(r"\"(.*?)\"", line[1]))
  keywords_mallet.append(out_line)


doc_reconstruction = []

for i, line in enumerate(raw_data):
    if line[0].split('_')[0] == interview_search_id:
        doc_reconstruction.append(i)
        
results = []

for i in doc_reconstruction:
    for tup in doc_tops_mallet[i]:
        if tup[0] < 10:
          y_label = '0'+str(keywords_mallet[tup[0]]).replace('"', '').replace("'", "").replace('[', '').replace(']', '').replace('(', '').replace(')', '') 
        else:
          y_label = str(keywords_mallet[tup[0]]).replace('"', '').replace("'", "").replace('[', '').replace(']', '').replace('(', '').replace(')', '') 
        results.append([y_label, int(raw_data[i][0].split('_')[-1]), tup[1]])
                        
df_heatmap = pd.DataFrame (results, columns = ['top', 'chunk ' + interview_search_id, 'weight'])
doc_tops_heatmap = df_heatmap.pivot("top", "chunk " + interview_search_id, "weight")
fig = px.imshow(doc_tops_heatmap, color_continuous_scale = 'emrld', width=breite, height=höhe,  aspect='auto')

fig.show()

#### Statische Version

In [ ]:
#@markdown Interview ID (suche nach ADG0001 etc.)

interview_search_id = 'ADG0019' #@param {type: 'string'}

#@markdown ##Grafik speichern?
save_fig = False #@param {type:"boolean"}
now = str(datetime.now())[:19]


if load_models == False:
  output = lda_model_mallet.print_topics(num_topics=topics, num_words=10)
  

if load_models == True:
  raw_data = load_dataset_mallet
  output = []
  for line in top_words_mallet:
    newline = ''
    for i in range (0, 10):
      newline = newline + line[1].split(' + ')[i] + ' '
    output.append([line[0], newline])

keywords_mallet = []

for line in output:
  out_line = line[0],str(re.findall(r"\"(.*?)\"", line[1]))
  keywords_mallet.append(out_line)




doc_reconstruction = []

for i, line in enumerate(raw_data):
    if line[0].split('_')[0] == interview_search_id:
        doc_reconstruction.append(i)
        
results = []

for i in doc_reconstruction:
    for tup in doc_tops_mallet[i]:
        #results.append([tup[0], raw_data[i][0], tup[1]])
        results.append([keywords_mallet[tup[0]], int(raw_data[i][0].split('_')[-1]), tup[1]])
        #results.append([keywords_mallet[tup[0]], int(raw_data[i][0].split()[-1]), tup[1]])
                        
df_heatmap = pd.DataFrame (results, columns = ['top', 'chunk ' + interview_search_id, 'weight'])
doc_tops_heatmap = df_heatmap.pivot("top", "chunk " + interview_search_id, "weight")
sns.set(rc = {'figure.figsize':(30,16)})
heatmap_out = sns.heatmap(doc_tops_heatmap, cmap="YlGnBu", square=False)
fig_out = heatmap_out.get_figure()

if save_fig:
  fig_out.savefig(output_path+interview_search_id+'_interview_topic_dispersion_'+name_dataset_mallet+'_'+str(topics_mallet)+'topics_'+ now +'.pdf')

###Textabschnitte ausgeben

Haben Sie ein interessantes Muster in der Heatmap lokalisiert oder sind auf anderem Wege auf einen Textabschnitt (=Chunk) aufmerksam geworden, können Sie hier einzelne, mehrere oder eine Reihe von Chunks gut lesbar ausgeben lassen. Mit >save_fig< können Sie den Auschnitt des Transkripts als Excel-Tabelle in den Ordner >WS MuM OUTPUT< speichern, den Sie in Ihrem Google Drive finden.

In [ ]:
import xlsxwriter



#@markdown ##Nummer der Chunks (X-Achse Heatmap)
#@markdown Kommaseparierte Aufzählung oder Range mit Bindestrich

chunk_nr = "134" #@param {type: 'string'}
chunk_nr = chunk_nr.replace(' ', '')

#@markdown ##Transkript speichern?
#@markdown Transkript wird als Excel-Sheet im Output-Ordner abgelegt

save_fig = True #@param {type:"boolean"}
now = str(datetime.now())[:19]

transcript = ''
transcript_out = []

if '-' in chunk_nr:
  chunk_range = [str(i) for i in range(int(chunk_nr.split('-')[0]),int(chunk_nr.split('-')[1]))]
  chunk_range.append(chunk_nr.split('-')[-1])

  for line in reference_dataset_mallet:
    if line[0][:7] == interview_search_id and line[0][7:] in chunk_range:
      transcript = transcript + line[0][:7] + ' Chunk ' + line[0][7:] + '\n\n'
      transcript_out.append([line[0][:7] + ' Chunk ' + line[0][7:]])
      lines = line[1].split('\n')
      for speaker_line in lines:
        transcript = transcript + speaker_line + '\n'
        transcript_out.append(speaker_line.split('\t'))


if ',' in chunk_nr:
  for line in reference_dataset_mallet:
    if line[0][:7] == interview_search_id and line[0][7:] in chunk_nr.split(','):
      transcript = transcript + line[0][:7] + ' Chunk ' + line[0][7:] + '\n\n'
      transcript_out.append([line[0][:7] + ' Chunk ' + line[0][7:]])
      lines = line[1].split('\n')
      for speaker_line in lines:
        transcript = transcript + speaker_line + '\n'
        transcript_out.append(speaker_line.split('\t'))

else:
  for line in reference_dataset_mallet:
    if line[0][:7] == interview_search_id and line[0][7:] == chunk_nr:
      transcript = transcript + line[0][:7] + ' Chunk ' + line[0][7:] + '\n\n'
      transcript_out.append([line[0][:7] + ' Chunk ' + line[0][7:]])
      lines = line[1].split('\n')
      for speaker_line in lines:
        transcript = transcript + speaker_line + '\n'
        transcript_out.append(speaker_line.split('\t'))

print(transcript)

if save_fig:
  workbook = xlsxwriter.Workbook(output_path + interview_search_id + '_Chunks_' + chunk_nr + '_' + name_dataset_mallet[:-20]+'_'+str(topics_mallet)+'topics_' + now + '.xlsx')   
  worksheet = workbook.add_worksheet()
  worksheet.set_column('A:B', 15)
  worksheet.set_column('B:C', 20)
  worksheet.set_column('C:D', 80)

  cell_format = workbook.add_format()
  cell_format.set_text_wrap()
  cell_format.set_align('left')
  cell_format.set_align('top')


  row = 0
  for line in transcript_out:
    if len(line) > 1:
 
      worksheet.write(row, 0, line[0], cell_format)
      worksheet.write(row, 1, line[1], cell_format)
      worksheet.write(row, 2, line[2], cell_format)
      row += 1
    if len(line) == 1:
      worksheet.write(row, 2, line[0])
      row += 2
  workbook.close()

ADG0058 Chunk 134

00:56:28.07	IP_OB	Wilhelm Krieger.
00:56:30.27	IP_OB	Und der war stark.
00:56:33.13	IP_OB	Und ein Schläger. Und mit dem habe ich auch Krieg geführt.
00:56:38.24	IP_OB	Da waren nachher in der Volksschule zwei große Gruppen.
00:56:42.06	IP_OB	Einmal die Gruppe Krieger und die Gruppe Büchler.
00:56:46.16	IP_OB	Im Spiel, da haben wir uns schon mal gegnseitig gebalgt.
00:56:51.02	IP_OB	Im Großen und Ganzen ging es aber friedlich ab.
00:56:54.11	IP_OB	Das war so ein Erlebnis aus der Volksschule.
00:56:57.08	IP_OB	Meine Mutter sagte immer zu mir, du bist viel zu gutmütig.
00:57:01.27	IP_OB	Meine Frau hat das auch gesagt, dass ich zu gutmütig bin.
00:57:05.04	IP_OB	Das bin ich auch in unserer Beziehung.
00:57:08.03	IP_OB	Ich helfe gerne, wenn es geht.
00:57:11.14	IP_OB	Aber was man eben kann. Ich (...) auch gerne. Das kommt auch gar nicht darauf an.
00:57:16.15	IP_OB	Das wissen hier die Kirche in, äh, die evangelische Kirche.
00:57:20.24	IP_OB	Ich habe viel in (Stiftel .) ge

##Suchroutinen

### Die am höchsten gewichteten Topics für ein Chunk ausgeben

In [ ]:
chunk_id = "ADG0042_7" #@param {type:"string"}

if load_models == True:
  raw_data = load_dataset_mallet

results = []

for i, line in enumerate(raw_data):
    if line[0] == chunk_id:
      for tup in doc_tops_mallet[i]:
        results.append((tup[1], tup[0]))

results_sorted = sorted(results, reverse=True)

for line in results_sorted[:10]:
  print('Topic '+str(line[1]) + ' (' + str(line[0]) + ')')

Topic 30 (0.3086074398646739)
Topic 18 (0.15285597917085506)
Topic 17 (0.11076256343615007)
Topic 3 (0.0904292690702355)
Topic 2 (0.045367676494454)
Topic 9 (0.04517813728383635)
Topic 49 (0.030855650423961857)
Topic 47 (0.02787713475746421)
Topic 26 (0.026030248809786653)
Topic 12 (0.02499728274451905)


### Nach Dokumenten mit einem bestimmten Topic suchen

Wenn sie gezielt nach Textabschnitten suchen möchten, in denen ein Topic dominiert, geben Sie bitte rechts die Topic-Nr ein, die Sie der Topic-Liste entnehmen können. Wählen Sie die Option >document_search<, können Sie die Ergebnisse auf eine Vorauswahl an Interviews eingrenzen. 

Der Threshold bestimmt, wie hoch das Gewicht eines Topics sein soll, damit es als Treffer gewertet wird. Sie können mit dem >threshold_custom_value< einen eigenen Wert zwischen 0 und 1 angeben, ansonsten wird automatisch der Mittelwert des Modells gesetzt. Sind die Ergebnisse zu zahlreich, erhöhen Sie das >threshold_topic_weight<, um nur höher gewichtete Textabschnitte zu erhalten und die Anzahl der Suchergebnisse einzugrenzen.

In [ ]:
#@markdown Nach welchem Topic (Nummer) soll gesucht werden?
topic_search =   22#@param {type:'integer'}

#@markdown Soll in einem bestimmten Dokument oder Chunk gesucht werden?
document_search = False #@param {type:"boolean"}
search_routine = "doc"

#@markdown Interview IDs kommasepariert eingeben
document_id = "ADG0011, ADG0002" #@param {type:"string"}
document_ids_repl = document_id.replace(' ', '')
document_ids = [id for id in document_ids_repl.split(',')]

#@markdown ##Threshold Topic-Weight
#@markdown If custom value is not checked, average topic weight is set as threshold.
threshold_custom_value = True #@param {type:"boolean"}
if threshold_custom_value == False:
  threshold_topic_weight = average_weight_mallet
if threshold_custom_value == True:
  threshold_topic_weight = 0.6 #@param {type:"number"}

if load_models == True:
  raw_data = load_dataset_mallet

for j, line in enumerate(doc_tops_mallet):
  for tup in line:
    if tup[0] == topic_search and tup[1] >= threshold_topic_weight:  
      if not document_search:  
        print(raw_data[j])
      if document_search:
        if search_routine == 'doc':
          if raw_data[j][0].split('_')[0] in document_ids:
            print(raw_data[j])    

['ADG0038_3', 'da lernte ich dann von denen dabei wie dieses holz wir nennen das ja dieses rundholz als kappe und stempel das obere ist die kappe und das andere ist der stempel dann hatten wir wenn das so in eck war von der deutschen türstockzimmerung was war das deutsche türstockzimmerung gerade nu haben se dat so schräg gestellt wegen dem druck wenn der jetzt zur seite drückt muss der ja nach oben gehen sonst kommt ne klar und da war ich jetzt irgendwo hier was wenn wir jetzt so vorbeifuhren zusammengerückt wurde so ecke wo wir abhauen mussten  ja  und wenn so schwerer druck war da kam dann draht drum son seil kam da drum von so nem so nem diese seile wo die rumgeschlagen mit so krampen damit das nicht so leicht wegbrechen konnte weil ohne schlag konnte es so leicht wegbrechen das war das erste was ich da so mitgemacht hatte und dann kam danach nachher wo steine schippen da lernte ich denn auch mit sprengstoff umgehen und die ersten bergleute die hatten dann ihren sprengstoff mussten

### Volltextsuche

Eine einfache Volltextsuche mit bis zu zwei Begriffen oder Wortfolgen und einem boolschen Operator.

In [ ]:
#@markdown Hier können einzelne Worte oder ngramme aus den obigen Listen 
#@markdown herauskopiert und eingefügt werden, um sie im Kontext zu lesen
suche1 = 'arbeit' #@param {type:'string'}
operator = 'and' #@param ['and', 'or']
suche2 = 'ruhrgebiet' #@param {type:'string'}
counter = 0
  
for i, doc in enumerate(raw_data):
  if operator == 'and':
    if suche1 in doc[1] and suche2 in doc[1]:
      counter += 1
      print(str(counter) + '. Treffer' + '\n\n' + str(i) + str(doc) + '\nTopic-Nr.: ' + str(doc_tops_mallet[i]) + '\n---------------------------------------------------------------------\n---------------------------------------------------------------------\n\n')
  if operator == 'or':
    if suche1 in doc[1] or suche2 in doc[1]:
      counter += 1
      print(str(counter) + '. Treffer' + '\n\n' + str(i) + str(doc) + '\nTopic-Nr.: ' + str(doc_tops_mallet[i]) + '\n---------------------------------------------------------------------\n---------------------------------------------------------------------\n\n')

print(str(counter) + ' Dokumente gefunden.')

# Model Training
Hier können neue Modelle trainiert werden.

##Installations & Import for Training New Models

In [ ]:
!pip install gensim==3.8.3
!pip install --upgrade spacy
!pip install https://github.com/explosion/spacy-models/releases/download/de_core_news_lg-3.2.0/de_core_news_lg-3.2.0.tar.gz
!apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
!unzip mallet-2.0.8.zip

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
os.environ['MALLET_HOME'] = '/content/mallet-2.0.8'
mallet_path = '/content/mallet-2.0.8/bin/mallet' # you should NOT need to change this

import spacy
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

##Load Dataset

In [ ]:
#@markdown #Dataset auswählen

files = 'LUSIR_df_speakers_clean_normalized_50sentence(s)_NEW - CORPUS' #@param ['LUSIR_df_speakers_clean_normalized_1sentence(s)_NEW - CORPUS', 'LUSIR_df_speakers_clean_normalized_5sentence(s)_NEW - CORPUS', 'LUSIR_df_speakers_clean_normalized_10sentence(s)_NEW - CORPUS', 'LUSIR_df_speakers_clean_normalized_25sentence(s)_NEW - CORPUS', 'LUSIR_df_speakers_clean_normalized_50sentence(s)_NEW - CORPUS']
dataframeimport = pd.read_pickle(path+files)  
name_dataset_gensim = files
name_dataset_mallet = files


modeldumps = output_path+"modeldumps/"

try:
    os.mkdir(modeldumps)
except OSError:
  print ("Directory %s already exists" % modeldumps)
else:
  print ("Successfully created the directory %s " % modeldumps)

raw_data = dataframeimport.values.tolist() # Umwandlung des Dataframes in eine Liste
print(raw_data[0])
print(len(raw_data))
data = [[word for word in line[1].split()] for line in raw_data] # Tokenisierung
print(data[0])
print(len(data))



##Load Stoplist

In [ ]:
open_stoplist = 'german_stopwords_full_BE_MOD Topics.txt'

stoplist = open(path+open_stoplist, encoding='UTF-16', mode='r').read().split()

print(stoplist)

##Preprocessing

### Stopwords entfernen

In [ ]:
#@markdown ###Choose Stopword Routine

stopword_routine = "curated_stoplist" #@param ["curated_stoplist", "stoplist_by_threshold"]

if stopword_routine == 'curated_stoplist':
  data_words_nostops = [[word for word in line if word not in stoplist] for line in data]
  print(data_words_nostops[0][:9])
  print(len(data_words_nostops))

if stopword_routine == 'stoplist_by_threshold':
  wordcounts = {}
  wordcount = 0

  #@markdown Threshold = Worthäufigkeit in Prozent
  threshold = 0.1 #@param {type:'number'}

  for line in data:
    wordcount = wordcount + len(line)

  for line in data:
    for word in line:
      if word not in wordcounts:
        wordcounts[word] = 1
      if word in wordcounts:
        wordcounts[word] += 1

  wordcounts_sorted = []

  for word, count in wordcounts.items():
    t = ((count/wordcount)*100, count, word)
    wordcounts_sorted.append(t)

  wordcounts_out = sorted(wordcounts_sorted, reverse=True)


  stoplist_by_threshold = [word[2] for word in wordcounts_out if word[0] > 0.1]

  data_words_nostops = [[word for word in line if word not in stoplist_by_threshold] for line in data]
  print(data_words_nostops[0][:9])

### Lemmatisierung

In [ ]:
def lemmatization(texts, allowed_postags=['NOUN', 'PROPN', 'VERBS', 'ADJ', 'ADV']):
  texts_out = []
  for sent in texts:
    doc = nlp(" ".join(sent))
    texts_out.append([token.lemma_ for token in doc])# if token.pos_ in allowed_postags])
  return texts_out

nlp = spacy.load('de_core_news_lg', disable=['parser', 'ner'])
allowed_postags=['NOUN', 'PROPN', 'VERBS', 'ADJ', 'ADV']
data_lemmatized = lemmatization(data_words_nostops, allowed_postags=allowed_postags) # hier können POS-Tags ein-/ausgeschlossen werden
#print('lemmata: ' + str(data_lemmatized[:9]))

# Lemmatisierte Daten erneut durch Stoplist schleifen
#@markdown Filter setzen für Mindestwortlänge (in Zeichen/Buchstaben)

min_wordlen = 2 #@param {type:"integer"}

data_final = [[word for word in doc if len(word) > min_wordlen and word not in stoplist] for doc in data_lemmatized] # Hier werden wörter mit weniger als drei Buchstaben ausgeschlossen
print(data_final[0][:9])
print(len(data_final))

###N-Grams

In [ ]:
def ngrams_topics(data, n):


    words = data
  
    if n == '2':
        a = 0
        b = 1
        bigrams = []
        counter = 0

        for doc in words:
          for word in doc:
 
            if counter < len(words) - 1:
                if words[a] in stoplist or words[b] in stoplist:
                    a = a + 1
                    b = b + 1
                    counter = counter + 1
                else:
                    x = words[a] + ' ' + words[b]
                    bigrams.append(x)
                    a = a + 1
                    b = b + 1
                    counter = counter + 1
        
        return bigrams


    if n == '3':
        a = 0
        b = 1
        c = 2
        trigrams = []
        counter = 0

        for doc in words:
          for word in doc:
            while counter < len(words) - 2:
                if words[a] and words[b] and words[c] in stoplist:
                    a = a + 1
                    b = b + 1
                    c = c + 1
                    counter = counter + 1
                else:
                    x = words[a] + ' ' + words[b] + ' ' + words[c]
                    trigrams.append(x)
                    a = a + 1
                    b = b + 1
                    c = c + 1
                    counter = counter + 1


        return trigrams

#@markdown #Auswählen: Bi- oder Trigramme
n = "2" #@param [2,3]

#@markdown # Dataset auswählen
#@markdown data = voller Datensatz, normalisiert

#@markdown data_words_nostops = stopwortbereinigt

#@markdown data_final = stopwortbereinigt und lemmatisiert

dataset_ngrams = "data_words_nostops" #@param ['data', 'data_words_nostops', 'data_final']

if dataset_ngrams == 'data':
  dataset_ngrams = data
if dataset_ngrams == 'data_words_nostops':
  dataset_ngrams = data_words_nostops
if dataset_ngrams == 'data_final':
  dataset_ngrams = data_final

data_ngrams = [ngrams_topics(line, n) for line in dataset_ngrams] # ohne Lemmatisierung: data, line[1]

freq = {}

for doc in data_ngrams:
  for ngram in doc:
    if ngram not in freq:
      freq[ngram] = 1
    else:
      freq[ngram] += 1

freq_sorted = []

for ngram, count in freq.items():
    t = (count, ngram)
    freq_sorted.append(t)

freq_final_1 = sorted(freq_sorted, reverse=True)

pprint(freq_final_1[:249])



In [ ]:
word_count = 0

dataset_wordcount = 'data_final' #@param ['data', 'data_words_nostops', 'data_final']

if dataset_wordcount == 'data':
  dataset_wordcount = data
if dataset_wordcount == 'data_words_nostops':
  dataset_wordcount = data_words_nostops
if dataset_wordcount == 'data_final':
  dataset_wordcount = data_final

for doc in dataset_wordcount:
  word_count = word_count + len(doc)
print(word_count)

##Training


In [ ]:
#@markdown ##Anzahl Topics 


topics =   50#@param {type:"integer"}
chunks = len(data)

#@markdown ##Dataset auswählen

name_dataset = "data_final" #@param ['data', 'data_words_nostops', 'data_final', 'data_ngrams']

if name_dataset == 'data':
  dataset = data
if name_dataset == 'data_words_nostops':
  dataset = data_words_nostops
if name_dataset == 'data_final':
  dataset = data_final
if name_dataset == 'data_ngrams':
  dataset = data_ngrams

id2word = corpora.Dictionary(dataset) 

corpus = [id2word.doc2bow(text) for text in dataset]

# Berechnung beider Modelle (Gensim, Mallet)
#@markdown ##Parametertuning

#@markdown ###Mallet

random_seed_mallet = 100 #@param {type:"integer"}
optimize_interval_mallet = 500 #@param {type:"integer"}
iterations_mallet = 5000 #@param {type:"integer"}

lda_model_mallet = gensim.models.wrappers.ldamallet.LdaMallet(mallet_path, corpus=corpus, id2word=id2word, 
  num_topics=topics, iterations=iterations_mallet, optimize_interval=optimize_interval_mallet, random_seed=random_seed_mallet
)

# Document-Topics-Liste erstellen und Topic-Weights berechnen

## Daten-Output Mallet konvertieren

doc_tops_import = open(lda_model_mallet.fdoctopics(), mode='r', encoding = 'UTF-8').read()

doc_tops_mallet = []
sum_top_weights = 0.0
top_counter = 0
min_weight_mallet = 1
max_weight_mallet = 0
for line in doc_tops_import.splitlines():
  doc_tops_transfer = []
  for topic_nr, topic in enumerate(line.split()):
    if '.' in topic:
      topic_float = float(topic)
      if topic_float >= 0: # Threshold für Weight
        sum_top_weights = sum_top_weights + topic_float
        top_counter += 1
        doc_tops_transfer.append((topic_nr - 2, topic_float)) # hier Weight als Float, in anderen Zellen als Str -> vereinheitlichen (?)
        if topic_float < min_weight_mallet:
          min_weight_mallet = topic_float
        if topic_float > max_weight_mallet:
          max_weight_mallet = topic_float        
  doc_tops_mallet.append(doc_tops_transfer)

average_weight_mallet = sum_top_weights/top_counter

print('Minimales Topic-Weight: ' + str(min_weight_mallet))
print('Durchschnittliches Topic-Weight: ' + str(average_weight_mallet))
print('Maximales Topic-Weight: ' + str(max_weight_mallet))

load_models = False
topics_mallet = topics

## Modelle speichern

Hier können die Dokument-Topic-Listen und Keyword-Listen der fertig berechneten Modelle abgespeichert werden. Für einen späteren Vergleich kann man die Modelle oben direkt abrufen und miteinander vergleichen, ohne jedesmal wieder neu trainieren zu müssen. Sämtliche Funktionen des Colabs laufen mit geladenen Daten.

In [ ]:
if load_models == True:
  print("Modell bereits gespeichert")

else:
  now = str(datetime.now())[:19]

  new_model_mallet = 'mallet_'+name_dataset_mallet+'_'+str(topics)+'topics_'+ now +'/'
  os.mkdir(modeldumps+new_model_mallet)
  doc_tops_mallet_df = pd.DataFrame(data=doc_tops_mallet)
  doc_tops_mallet_df.to_pickle(modeldumps+new_model_mallet+'mallet_'+name_dataset_mallet+'_'+str(topics)+'topics_'+now +'.doc_tops_mallet')
  top_words_mallet_df = pd.DataFrame(data=lda_model_mallet.print_topics(num_topics=topics, num_words=1000))
  top_words_mallet_df.to_pickle(modeldumps+new_model_mallet+'mallet_'+name_dataset_mallet+'_'+str(topics)+'topics_'+now +'.top_words_mallet')
  out = open(modeldumps+new_model_mallet+'mallet_'+name_dataset_mallet+'_'+str(topics)+'topics_'+now +'.txt', 'w', encoding='UTF-8')
  out.write(name_dataset_mallet + '\n')
  out.write('Anzahl Topics: ' + str(topics) + '\n')
  if name_dataset == 'data_final':
    out.write('Stopwords: ' + open_stoplist + '\n')
    out.write('Lemmatisiert, Pos-Tag-Filter: ' + str(allowed_postags) + '\n')
  if name_dataset == 'data_words_nostops':
    out.write('Stopwords: ' + open_stoplist + '\n')
  if name_dataset == 'data':
    out.write('Kein Data Cleaning durchgeführt' + '\n')
  out.write('random_seed_mallet: ' + str(random_seed_mallet) + '\n')
  out.write('optimiize_interval_mallet: ' + str(optimize_interval_mallet) + '\n')
  out.write('iterations_mallet: ' + str(iterations_mallet) + '\n')
  out.close()

## Topword Diversity
Hier soll die Einzigartigkeit von Keywords innerhalb der Topics untersucht werden, in je weniger Topics ein Wort enthalten ist, desto trennschärfer sollten die Topics sein. Vgl. mit der Relevance Metric bei pyLDAvis, nähere Infos dazu im oben verlinkten Paper, und dem Exclusivity Score im Paper von Thompson/Mimno. Bislang harter Filter, es werden NUR einzigartige Wörter gezählt.

In [ ]:
#Topword diversity
number_of_words =  30#@param {type:"integer"}

if load_models == False:
  topwords = []
  toptops = lda_model_mallet.print_topics(num_topics=topics, num_words=number_of_words)
  for tupel in toptops:
    words = re.findall(r"\"(.*?)\"", tupel[1])
    for word in words:
      topwords.append(word)

if load_models == True:
    topwords = []
    for line in top_words_mallet:
      newline = []
      for i in range (0, number_of_words):
        newline.append(re.findall(r"\"(.*?)\"", line[1].split(' + ')[i])[0])
      for word in newline:
        topwords.append(word)

print(len(topwords))
print(len(set(topwords)))
singularity = (len(set(topwords)) / (int(topics_mallet) * number_of_words)) * 100
print(str(int(singularity))+'%')


## Größte und kleinste Topics

In [ ]:
#@markdown ##Gesamt-Topic-Weights als Liste speichern?
save_doc = False #@param {type:"boolean"}

#@markdown #Grafik speichern?
save_fig = False #@param {type:"boolean"}

top_weight_sum = {}

for i, line in enumerate(doc_tops_mallet):
  for tup in line:
    if tup[0] not in top_weight_sum:
      top_weight_sum[tup[0]] = tup[1]
    if tup[0] in top_weight_sum:
      top_weight_sum[tup[0]] += tup[1]

points = []
for top, weight in top_weight_sum.items():
  points.append((top, weight))

weights_reverse = [(tup[1], tup[0]) for tup in points]
weights_sorted = sorted(weights_reverse, reverse=True)
print(weights_sorted)

if save_doc:
  out = open(output_path+'Mallet_top_weight_sums_'+name_dataset_mallet+'_'+str(topics_mallet)+'topics_'+ now +'.txt', 'w', encoding='UTF-8')
  for line in weights_sorted:
    out.write(str(line))
  out.close()

x, y = list(zip(*points))
pylab.bar(x, y)
pylab.title('Topic Weight Sums')
pylab.ylabel('Sum Weights')
pylab.xlabel("Topic Nr.")
if save_fig:
  pylab.savefig(output_path+'Mallet_topic_weight_sums_'+ name_dataset_mallet + '_' + str(topics_mallet) + 'topics'+ now +'.pdf')
pylab.show() 

print(points)